<h2>MODEL TESTING</h2>

Script ini digunakan hanya untuk mengetes model-model yang telah dibuat. Tahapan dari mengetes suatu model adalah:
1. Load model yang telah ada. (Saat ini, model yang diloat adalah model v1 yang sudah saya buat).
2. Load x_test dan y_test, untuk menjadi data yang dapat dimasukkan ke model untuk melihat performa model. Bagian ini tidak perlu diubah jika model yang diload berubah.
3. Jalankan block terakhir. Num dapat diubah-ubah untuk melihat trajectory yang berbeda-beda

In [11]:
import tensorflow as tf
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [19]:
## Load the model
model = tf.keras.models.load_model('../models/model_v2.keras')

# Show the model architecture
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 20, 6)          │            13 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 20, 128)        │        69,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 20, 128)        │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 750,741 (2.86 MB)

 Trainable params: 250,242 (977.51 KB)

 Non-trainable params: 13 (56.00 B)

 Optimizer params: 500,486 (1.91 MB)

In [13]:
## Load numpy test data
x_test = np.load('../x_test_B.npy')
y_test = np.load('../y_test_B.npy')
mean_variance = np.load('../mean_variance_B.npy')

In [14]:
## IMPORTANT: Set up denormalizer
mean = mean_variance[0]
variance = mean_variance[1]

denormalizer = tf.keras.layers.Normalization(mean=mean, variance=variance, invert=True)

In [15]:
# Num dapat diubah-ubah untuk melihat trajectory yang berbeda-beda
num = 67

# Predict the next coords of entry "num" using the model
num_of_timesteps = len(x_test[0])
pred_coords = denormalizer(model.predict(x_test[num].reshape(1, num_of_timesteps, 6)))
true_coords = denormalizer(y_test[num])

# Print the true and predicted coordinates
print("Prediction:")
print(pred_coords)

print("True Coordinate:")
print(true_coords)


## Map display of True Point vs Predicted Point
past_lat = []
past_lon = []
past_speed_bearing = []

# Get route coordinates
for point in x_test[num]:
  past_lat.append(point[0])
  past_lon.append(point[1])
  past_speed_bearing.append([point[2], point[3]])

# Get prediction coordinates
pred_lat = pred_coords.numpy()[0][0]
pred_lon = pred_coords.numpy()[0][1]
pred_speed_bearing = [pred_coords.numpy()[0][2], pred_coords.numpy()[0][3]]

# Get true coordinates from y
true_lat = true_coords.numpy()[0][0]
true_lon = true_coords.numpy()[0][1]
true_speed_bearing = [true_coords.numpy()[0][2], true_coords.numpy()[0][3]]

# Build the map
fig = go.Figure()

fig.add_trace(go.Scattermapbox(
  mode="markers+lines",
  lat=past_lat,
  lon=past_lon,
  name="trip",
  hoverinfo='text',
  text=past_speed_bearing,
  marker={'size': 10}))

fig.add_trace(go.Scattermapbox(
  mode="markers",
  lat=[pred_lat],
  lon=[pred_lon],
  name="Predicted Point",
  hoverinfo='text',
  text=[pred_speed_bearing],
  marker={'size': 15, 'color': 'green'}))

fig.add_trace(go.Scattermapbox(
  mode="markers",
  lat=[true_lat],
  lon=[true_lon],
  name="True Point",
  hoverinfo='text',
  text=[true_speed_bearing],
  marker={'size': 15, 'color': 'yellow'}))

fig.update_layout(mapbox_style="open-street-map",)
fig.update_layout(
  margin={"r":0,"t":0,"l":0,"b":0},
  mapbox=dict(
    zoom=10,
    center=go.layout.mapbox.Center(
            lat=-6.2,
            lon=106.816
        ),
  ))
fig.show()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step


InvalidArgumentError: {{function_node __wrapped__Reshape_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input to reshape is a tensor with 4 values, but the requested shape has 2 [Op:Reshape]

In [ ]:
## MULTI STEP PREDICTION
# Num dapat diubah-ubah untuk melihat trajectory yang berbeda-beda
num = 234
steps = 60

# Predict the next "step" coords of entry "num" using the model
num_of_timesteps = len(x_test[0])
x_data = x_test[num] # Data that will be fed to the model

pred_coords = []
for i in range(steps):
  current_pred = denormalizer(model.predict(x_data.reshape(1, num_of_timesteps, 6))).numpy().tolist()[0]
  pred_coords.append(current_pred)

  x_data = np.delete(x_data, 0, 0)

  hour = x_data[-1][4]
  day = x_data[-1][5]

  current_pred.extend((hour, day))
  
  x_data = np.append(x_data, [current_pred], axis=0)

true_coords = denormalizer(y_test[num])

# Print the true and predicted coordinates
print("Prediction:")
print(pred_coords)

print("True Coordinate:")
print(true_coords)


### Map display of True Point vs Predicted Point
# Get route coordinates
past_lat = []
past_lon = []
past_speed_bearing = []
for point in x_test[num]:
  past_lat.append(point[0])
  past_lon.append(point[1])
  past_speed_bearing.append([point[2], point[3]])

# Get prediction coordinates
pred_lat = []
pred_lon = []
pred_speed_bearing = []
for point in pred_coords:
  pred_lat.append(point[0])
  pred_lon.append(point[1])
  pred_speed_bearing.append([point[2], point[3]])

# Get true coordinates from y
true_lat = true_coords.numpy()[0][0]
true_lon = true_coords.numpy()[0][1]
true_speed_bearing = [true_coords.numpy()[0][2], true_coords.numpy()[0][3]]


# Build the map
fig = go.Figure()

# Plot previous path
fig.add_trace(go.Scattermapbox(
  mode="markers+lines",
  lat=past_lat,
  lon=past_lon,
  name="trip",
  hoverinfo='text',
  text=past_speed_bearing,
  marker={'size': 10}))

# Plot predicted path
fig.add_trace(go.Scattermapbox(
  mode="markers+lines",
  lat=pred_lat,
  lon=pred_lon,
  name="Predicted Point",
  hoverinfo='text',
  text=pred_speed_bearing,
  marker={'size': 10, 'color': 'green'}))

fig.add_trace(go.Scattermapbox(
  mode="markers",
  lat=[true_lat],
  lon=[true_lon],
  name="True Point",
  hoverinfo='text',
  text=true_speed_bearing,
  marker={'size': 15, 'color': 'yellow'}))

fig.update_layout(mapbox_style="open-street-map",)
fig.update_layout(
  margin={"r":0,"t":0,"l":0,"b":0},
  mapbox=dict(
    zoom=10,
    center=go.layout.mapbox.Center(
            lat=-6.2,
            lon=106.816
        ),
  ))
fig.show()

ValueError: Exception encountered when calling Normalization.call().

[1mDimensions must be equal, but are 6 and 2 for '{{node sequential_1/normalization_1/Sub}} = Sub[T=DT_FLOAT](data, sequential_1/normalization_1/Sub/y)' with input shapes: [1,20,6], [1,1,2].[0m

Arguments received by Normalization.call():
  • inputs=tf.Tensor(shape=(1, 20, 6), dtype=float32)